<a href="https://colab.research.google.com/github/119020/NLP_2025_Spring_Materials/blob/main/25Spring_NLP_Assignment_2_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Enginnering
### **Course Name:** Natural Language Processing **<font color="red">(CSC6052/MDS5110/CSC5051/CSC4100)</font>**




Hello, everyone.
In this tutorial, we'll explore *Prompt Engineering* techniques.

**<font color="blue">What is prompt engineering in AI?</font>**

An AI prompt is a carefully crafted instruction given to an AI model to generate a specific output. These inputs can range from text and images to videos or even music.

Prompt engineering means writing precise instructions that guide AI models like **ChatGPT** to produce specific and useful responses. It involves designing inputs that an AI can easily understand and act upon, ensuring the output is relevant and accurate.

## **The First Step: Mastering the Use of APIs**



### About API Keys
#### DeepSeek API Keys
❗ Note that you can get your deepseek api key from it official website. Upon registeration, you will get 14 RMB free credit, and it will account for about 20M token usage, which will be enough to cover your assignment. Alternatively, you can go to [Siliconflow](https://cloud.siliconflow.cn/) for more free credit:
- Go to the website [https://platform.deepseek.com/api_keys](https://platform.deepseek.com/api_keys).
- Sign up for a new account, and you will have 14 RMB free credit.
- Change the `DEEPSEEK_API_KEY` environment variable with the key you purchased.
- Remember to update `DEEPSEEK_BASE_URL` to https://api.siliconflow.cn/v1/chat/completions when using API from SiliconFlow.

#### OpenAI API Keys
Note that we provide a key with 100 US dollars, if it is used up you need to buy the Keys yourself (it may cost you a little bit of money), here is how to buy the keys:
- Go to the website [https://eylink.cn/buy/7](https://eylink.cn/buy/7).
- Purchase a 14 RMB key (10 US dollars). (10 dollars are enough.)
- Fill in the `OPENAI_API_KEY` below with the key you purchased.

(As a student, you can apply for a $100 free API credit at https://azure.microsoft.com/en-us/free/students. Remember to update `OPENAI_BASE_URL` to https://api.openai.com/v1/chat/completions when using API from Azure.)

🔅 To facilitate easier access to OpenAI's model APIs, we make use of a popular framework langchain.

In [4]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-deepseek
!pip install retrying

In [6]:
import os
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import random
import json
from retrying import retry
import requests


os.environ["DEEPSEEK_API_KEY"] = "sk-76b03e5991db4845836cb45121a63bcf"
os.environ["DEEPSEEK_BASE_URL"] = "https://api.deepseek.com/v1"
deepseek_chat = ChatDeepSeek(model="deepseek-chat", temperature=0)



😊 You can now engage directly with DeepSeek-Chat using our `invoke` function.

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant, please answer user's question."),
        ("user", "{input}")
    ]
)

#model = ChatDeepSeek(model="deepseek-chat")

chain = prompt | deepseek_chat

response = chain.invoke({"input": "Hello"})
print(response)


content='Hello! How can I assist you today? 😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 16, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 16}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '01e53780-f8fb-4a27-89bb-32682a3b9004', 'finish_reason': 'stop', 'logprobs': None} id='run-65790484-e524-4359-8057-f18e0cb1b78a-0' usage_metadata={'input_tokens': 16, 'output_tokens': 11, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


To make the output easy to use, we can apply a output parser to the original output!

In [8]:
chain = prompt | deepseek_chat | StrOutputParser()

response = chain.invoke({"input": "Hello"})
print(response)

Hello! How can I assist you today? 😊


### **LLM Setting**

**1. Model Selection**

You are free to choose from several models. Below are the models DeepSeek offers:
- deepseek-chat (currently pointing to deepseek-v3, offering a balance of capability and cost, highly recommended!)
- deepseek-reasoner (currently pointing to deepseek-r1, extremely sophisticated and intelligent)
For detailed information, please visit [DeepSeek's Website](https://api-docs.deepseek.com/zh-cn/news/news250120).

Note that DeepSeek offers a discount when using API in the midnight, details showing [here](https://api-docs.deepseek.com/zh-cn/quick_start/pricing)

Below are the models OpenAI offers:

- gpt-4o-mini (the most cost-effective model, highly recommended!)
- gpt-4o (offers a balance of capability and cost)
- gpt-4-turbo (extremely sophisticated and intelligent)

For detailed information, please visit [OpenAI's Website](https://platform.openai.com/docs/models/gpt-4-turbo-and-gpt-4).

**2. Temperature**
Controls the randomness of the model's output. Lower values make responses more deterministic and focused, while higher values allow for more creative and diverse outputs. Use low values for factual tasks and high values for creative tasks.

**3. Top P**
This is nucleus sampling, where only tokens from the top probability mass (up to `top_p`) are considered. Lower values encourage more focused responses, while higher values increase the diversity of possible outputs.

**4. Max Length** This limits the total number of tokens the model can generate, helping control response length and prevent irrelevant output.

### **Prompting Techniques**

**1. Zero-Shot Prompting** Large language models (LLMs) today, such as GPT-3.5 Turbo, GPT-4, and Claude 3, are tuned to follow instructions and are trained on large amounts of data. Large-scale training makes these models capable of performing some tasks in a "zero-shot" manner. Zero-shot prompting means that the prompt used to interact with the model won't contain examples or demonstrations. The zero-shot prompt directly instructs the model to perform a task without any additional examples to steer it.

In [9]:
#model = ChatDeepSeek(model="deepseek-chat")

chain = prompt | deepseek_chat | StrOutputParser()

your_prompt = """Classify the text into neutral, negative or positive.
Text: I think the vacation is okay.
Sentiment:"""

chain.invoke({"input": your_prompt})

'The sentiment of the text "I think the vacation is okay." is **neutral**.  \n\nThe word "okay" indicates a neither strongly positive nor negative feeling, suggesting a moderate or indifferent sentiment.'

**2. Few-Shot Prompting** While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

According to [Touvron et al. 2023](https://arxiv.org/pdf/2302.13971.pdf) few shot properties first appeared when models were scaled to a sufficient size ([Kaplan et al., 2020](https://arxiv.org/abs/2001.08361)).

In [10]:
your_prompt = """This is bad! // Negative
This is awesome! // Positive
Wow that movie was rad! // Positive
What a horrible show! //"""
chain.invoke({"input": your_prompt})

'Negative'



**3. Chain-of-Thought Prompting** Introduced in [Wei et al. (2022)](https://arxiv.org/abs/2201.11903), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

![](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fcot.1933d9fe.png&w=1920&q=75)  

In [11]:
your_prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
Let's think step by step."""
chain.invoke({"input": your_prompt})

"Let's break down the problem step by step to find out how many apples you remain with.\n\n1. **Initial Purchase:**\n   - You bought **10 apples**.\n   - **Total apples now:** 10\n\n2. **Giving Apples to Others:**\n   - You gave **2 apples** to the neighbor.\n   - You gave **2 apples** to the repairman.\n   - **Total apples given away:** 2 (neighbor) + 2 (repairman) = **4 apples**\n   - **Apples remaining after giving away:** 10 - 4 = **6 apples**\n\n3. **Second Purchase:**\n   - You bought **5 more apples**.\n   - **Total apples now:** 6 (remaining) + 5 (new) = **11 apples**\n\n4. **Eating an Apple:**\n   - You ate **1 apple**.\n   - **Apples remaining after eating:** 11 - 1 = **10 apples**\n\n**Final Answer:** You remain with \\(\\boxed{10}\\) apples."

## **Task 1: LLMs as a knowledgeable doctor**




The pharmacist licensure exam  is a cornerstone in the pharmacy profession, ensuring that candidates possess the requisite knowledge and skills for safe and effective practice. Its significance lies not only in validating credentials but also in safeguarding public health, enabling professional recognition, and ensuring adherence to legal and regulatory standards.

Advanced models like ChatGPT have significant potential in exam preparation, boasting an extensive knowledge base and the capability to provide in-depth explanations and clarify complex concepts. However, despite the prowess of such large models, if prompts are not designed appropriately, the information retrieved might be inaccurate or incomplete, potentially hindering success in the pharmacist exam.

In [15]:
!wget https://NLP-course-cuhksz.github.io/Assignments/Assignment2/task1/data/1.exam.json

--2025-03-27 10:27:10--  https://nlp-course-cuhksz.github.io/Assignments/Assignment2/task1/data/1.exam.json
Resolving nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86223 (84K) [application/json]
Saving to: ‘1.exam.json.1’

1.exam.json.1       100%[===================>]  84.20K  --.-KB/s    in 0.02s   

2025-03-27 10:27:10 (3.47 MB/s) - ‘1.exam.json.1’ saved [86223/86223]



In [16]:
import json
with open('1.exam.json') as f:
  data = json.load(f)

In [61]:
data[24]

{'question': '63. 表⽰药物在体内波动程度的参数是（）。',
 'option': {'A': 'Cmax', 'B': 'DE', 'C': 'MRT', 'D': 'AUMC', 'E': 'AUC'},
 'analysis': '药物在体内波动程度的参数是峰浓度（C ），反应药物吸收的程度，也与吸收速度有关。max',
 'answer': 'A',
 'question_type': '配伍选择题',
 'source': '2021年执业药师职业资格考试《药学专业知识（⼀）》',
 'options': 'A:Cmax\nB:DE\nC:MRT\nD:AUMC\nE:AUC',
 'deepseek_ans': 'B'}

In [21]:
your_prompt = """请回答下面的多选题，请直接正确答案选项，不要输出其他内容。
{question}
{options}"""

def get_query(da):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return your_prompt.format_map(da)

get_query(data[0])

'请回答下面的多选题，请直接正确答案选项，不要输出其他内容。\n27. 根据国家药品监督管理局，公安部，国家卫⽣健康委员会的有关规定，⼜服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他⿇醉药品，精神药品或者药品类易制毒化学品的复⽅制剂列⼊（）。\nA:含⿇醉药品复⽅制剂的管理\nB:第⼆类精神药品管理\nC:第⼀类精神药品管理\nD:医疗⽤毒性药品管理\nE:'

In [22]:
chain.invoke(get_query(data[0]))

'B:第⼆类精神药品管理'

In [24]:
# Zero-shot 计算做题准确率
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant, please answer user's question."),
        ("user", "{input}")
    ]
)
deepseek_chat = ChatDeepSeek(model="deepseek-chat", temperature=0)
chain = prompt | deepseek_chat | StrOutputParser()
your_prompt = """Please answer the multiple choice questions below, please direct the correct answer option and do not output anything else.
{question}
{options}"""

import re
from tqdm import tqdm

def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

correct_num = 0
total_num = 0
#for da in tqdm(data[:10]):
for da in tqdm(data):
  print(total_num+1)
  da['deepseek_ans'] =  chain.invoke(get_query(da))
  print(da['answer'])
  print(da['deepseek_ans'])
  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
  total_num += 1
print(f"模型准确率: {correct_num/total_num:.2%}")


  0%|          | 0/100 [00:00<?, ?it/s]

1


  1%|          | 1/100 [00:04<08:03,  4.88s/it]

B
B:第⼆类精神药品管理
2


  2%|▏         | 2/100 [00:09<08:03,  4.94s/it]

...

B
A
99


 99%|█████████▉| 99/100 [08:21<00:04,  4.35s/it]

B
B
100


100%|██████████| 100/100 [08:25<00:00,  5.06s/it]

E
E
模型准确率: 79.00%


In [45]:
!wget https://NLP-course-cuhksz.github.io/Assignments/Assignment2/task1/data/1.exam_demo.json
with open('1.exam_demo.json') as f:
  sample = json.load(f)

sample[4]


--2025-03-27 11:36:13--  https://nlp-course-cuhksz.github.io/Assignments/Assignment2/task1/data/1.exam_demo.json
Resolving nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6979 (6.8K) [application/json]
Saving to: ‘1.exam_demo.json.6’

1.exam_demo.json.6  100%[===================>]   6.82K  --.-KB/s    in 0s      

2025-03-27 11:36:13 (68.6 MB/s) - ‘1.exam_demo.json.6’ saved [6979/6979]



{'question': '5. 关于药品上市许可持有⼈委托储存、运输⾏为管理要求的说法，正确的是（）。',
 'option': {'A': '药品上市许可持有⼈应当将受托⽅储存、运输、销售等⾏为纳⼊⼰⽅质量管理体系，与其签订委托协议，约定双⽅药品质量责任，并对受托⽅进⾏监督',
  'B': '接受委托储存、运输药品的企业表明不知道承运承储的药品已超过有效期，⽽为其提供储存、运输服务的，可以免予⾏政处罚',
  'C': '接受疫苗上市许可持有⼈委托储存、运输的企业，不得再次委托储存、运输疫苗；不得将疫苗与其他药品混库储存或者混车、混箱运输',
  'D': '药品上市许可持有⼈应当对受托⽅的质量保证能⼒及风险管理能⼒进⾏评估，确认受托⽅符合《药品经质量管理规范》有关储存、运输的相关要求',
  'E': ''},
 'analysis': 'D项，药品上市许可持有⼈委托储存、运输药品的，应当对受托⽅的质量保证能⼒和风险管理能⼒进⾏评估，与其签订委托协议，约定药品质量责任、操作规程等内容，并对受托⽅进⾏监督。',
 'answer': 'D',
 'question_type': '最佳选择题',
 'source': '2021年执业药师职业资格考试《药事管理与法规》'}

In [58]:
sample_prompt = """Give the following sample question:
{question}
{options}
Answer:{answer}
Analysis:{analysis}"""

def get_sample(da):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return sample_prompt.format_map(da)

get_sample(sample[4])

'Give the following sample question:\n5. 关于药品上市许可持有⼈委托储存、运输⾏为管理要求的说法，正确的是（）。\nA:药品上市许可持有⼈应当将受托⽅储存、运输、销售等⾏为纳⼊⼰⽅质量管理体系，与其签订委托协议，约定双⽅药品质量责任，并对受托⽅进⾏监督\nB:接受委托储存、运输药品的企业表明不知道承运承储的药品已超过有效期，⽽为其提供储存、运输服务的，可以免予⾏政处罚\nC:接受疫苗上市许可持有⼈委托储存、运输的企业，不得再次委托储存、运输疫苗；不得将疫苗与其他药品混库储存或者混车、混箱运输\nD:药品上市许可持有⼈应当对受托⽅的质量保证能⼒及风险管理能⼒进⾏评估，确认受托⽅符合《药品经质量管理规范》有关储存、运输的相关要求\nE:\nanswer:D\nanalysis:D项，药品上市许可持有⼈委托储存、运输药品的，应当对受托⽅的质量保证能⼒和风险管理能⼒进⾏评估，与其签订委托协议，约定药品质量责任、操作规程等内容，并对受托⽅进⾏监督。'

In [59]:
# Few-shot 计算做题准确率
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant, please answer user's question."),
        ("user", "{input}")
    ]
)
deepseek_chat = ChatDeepSeek(model="deepseek-chat", temperature=0)
chain = prompt | deepseek_chat | StrOutputParser()
your_prompt = get_sample(sample[4]) + "\n\n" + """Please answer the multiple choice questions below, please direct the correct answer option and do not output anything else.
{question}
{options}"""

import re
from tqdm import tqdm

def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

correct_num = 0
total_num = 0
#for da in tqdm(data[:10]):
for da in tqdm(data):
  print(da)
  print(total_num+1)
  da['deepseek_ans'] = chain.invoke(get_query(da))
  print(da['answer'])
  print(da['deepseek_ans'])
  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
  total_num += 1
print(f"模型准确率: {correct_num/total_num:.2%}")


  0%|          | 0/100 [00:00<?, ?it/s]

{'question': '27. 根据国家药品监督管理局，公安部，国家卫⽣健康委员会的有关规定，⼜服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他⿇醉药品，精神药品或者药品类易制毒化学品的复⽅制剂列⼊（）。', 'option': {'A': '含⿇醉药品复⽅制剂的管理', 'B': '第⼆类精神药品管理', 'C': '第⼀类精神药品管理', 'D': '医疗⽤毒性药品管理', 'E': ''}, 'analysis': '⼜服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他⿇醉药品、精神药品或药品类易制毒化学品的复⽅制剂列⼊第⼆类精神药品管理。', 'answer': 'B', 'question_type': '最佳选择题', 'source': '2021年执业药师职业资格考试《药事管理与法规》', 'options': 'A:含⿇醉药品复⽅制剂的管理\nB:第⼆类精神药品管理\nC:第⼀类精神药品管理\nD:医疗⽤毒性药品管理\nE:', 'deepseek_ans': '根据国家药品监督管理局、公安部、国家卫生健康委员会的相关规定，口服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他麻醉药品、精神药品或者药品类易制毒化学品的复方制剂列入**第二类精神药品管理**。\n\n**正确答案：B**'}
1


  1%|          | 1/100 [00:04<07:55,  4.80s/it]

B
B
{'question': '112. 患者，男，70岁，因抑郁症长期服⽤舍曲林。新诊断为帕⾦森病，宜选⽤的药物有（）。', 'option': {'A': '司来吉兰', 'B': '普拉克索', 'C': '⾦刚烷胺', 'D': '吡贝地尔', 'E': '恩他卡朋'}, 'analysis': '上述题⼲中患者使⽤的抑郁药舍曲林是选择性5-羟⾊胺再摄取抑制剂，司来吉兰避免与选择性5-羟⾊胺再摄取抑制剂合⽤，因此答案选BCDE。', 'answer': 'BCDE', 'question_type': '多项选择题', 'source': '2021年执业药师职业资格考试《药学综合知识与技能》', 'options': 'A:司来吉兰\nB:普拉克索\nC:⾦刚烷胺\nD:吡贝地尔\nE:恩他卡朋', 'deepseek_ans': 'B:普拉克索'}
2


  2%|▏         | 2/100 [00:09<07:45,  4.75s/it]

...

B
B
{'question': '72. 多数脂溶性⼩分⼦类药物的吸收机制是（）。', 'option': {'A': '膜孔滤过', 'B': '简单扩散', 'C': '膜动转运', 'D': '易化扩散', 'E': '主动转运'}, 'analysis': '简单扩散：⽣物膜为类脂双分⼦层，脂溶性药物可以溶于脂质膜中，容易穿过细胞膜。', 'answer': 'B', 'question_type': '配伍选择题', 'source': '2021年执业药师职业资格考试《药学专业知识（⼀）》', 'options': 'A:膜孔滤过\nB:简单扩散\nC:膜动转运\nD:易化扩散\nE:主动转运', 'deepseek_ans': 'B:简单扩散'}
99


 99%|█████████▉| 99/100 [07:59<00:05,  5.21s/it]

B
B
{'question': '48. 患者，男，54岁，因痛风服⽤⾮布司他，为增强降尿酸效果，可同时使⽤的药物是（）。', 'option': {'A': '阿托伐他汀', 'B': '甲巯咪唑', 'C': '丙硫氧嘧啶', 'D': '氢氯噻嗪', 'E': '碳酸氢钠'}, 'analysis': '碳酸氢钠是⼀种抗酸剂，⼜服后可迅速中和胃酸，解除胃酸过多或烧⼼症状。此外，它也有⼀定的碱化尿液的效果。所以，临床上往往将碳酸氢钠结合⾮布司他等抑制尿酸⽣成的药物合⽤，碱化尿液，提⾼降尿酸的效果。', 'answer': 'E', 'question_type': '配伍选择题', 'source': '2021年执业药师职业资格考试《药学综合知识与技能》', 'options': 'A:阿托伐他汀\nB:甲巯咪唑\nC:丙硫氧嘧啶\nD:氢氯噻嗪\nE:碳酸氢钠', 'deepseek_ans': 'E:碳酸氢钠'}
100


100%|██████████| 100/100 [08:03<00:00,  4.84s/it]

E
E
模型准确率: 79.00%


<font color="blue">You need to optimize the prompt to improve the performance (accuracy) of large language models (LLMs).</font>

In [25]:
# CoT 计算做题准确率
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant, please answer user's question."),
        ("user", "{input}")
    ]
)
deepseek_chat = ChatDeepSeek(model="deepseek-chat", temperature=0)
chain = prompt | deepseek_chat | StrOutputParser()
your_prompt = """你是一名严谨的药剂学专家，请按以下步骤处理问题：
1. 分析题干关键词
2. 调用检索工具验证医学依据
3. 综合所有信息选择最准确的答案
4. 用中文输出最终选项
Please answer the multiple choice questions below, please direct the correct answer option and do not output anything else.
{question}
{options}"""

import re
from tqdm import tqdm

def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

correct_num = 0
total_num = 0
#for da in tqdm(data[:10]):
for da in tqdm(data):
  print(total_num+1)
  da['deepseek_ans'] =  chain.invoke(get_query(da))
  print(da['answer'])
  print(da['deepseek_ans'])
  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
  total_num += 1
print(f"模型准确率: {correct_num/total_num:.2%}")

  0%|          | 0/100 [00:00<?, ?it/s]

1


  1%|          | 1/100 [00:08<14:44,  8.93s/it]

B
根据国家药品监督管理局、公安部、国家卫生健康委员会的相关规定，口服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他麻醉药品、精神药品或者药品类易制毒化学品的复方制剂列入**第二类精神药品管理**。

**正确答案：B**
2


  2%|▏         | 2/100 [00:13<10:17,  6.30s/it]

...

B
A:国家医疗保障局
99


 99%|█████████▉| 99/100 [12:38<00:05,  5.72s/it]

B
B:简单扩散
100


100%|██████████| 100/100 [12:43<00:00,  7.63s/it]

E
E:碳酸氢钠
模型准确率: 79.00%
